<a href="https://colab.research.google.com/github/Samridhi0705/MajorProject/blob/main/mp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install telethon nest_asyncio

import nest_asyncio
nest_asyncio.apply()


In [ ]:
import os
import csv
from telethon import TelegramClient, events
from datetime import datetime

# Remove old session to force fresh login each time
if os.path.exists('bot_session.session'):
    os.remove('bot_session.session')

api_id = '28932578'
api_hash = 'e73796fdffd4d3fa973e1aedbe803311'
bot_token = '8315747892:AAHpfpYvvuC9lXBpycHHOBarC8_DTvOV0fs'  # Your bot token

client = TelegramClient('bot_session', api_id, api_hash)

# CSV file name
csv_file = 'flagged_messages.csv'

# Initialize CSV file with headers (only once)
if not os.path.exists(csv_file):
    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['timestamp', 'sender_id', 'message'])

async def main():
    await client.start(bot_token=bot_token)

    @client.on(events.NewMessage)
    async def handler(event):
        msg = event.message.message
        sender = await event.get_sender()
        sender_id = sender.id if sender else 'Unknown'
        timestamp = event.message.date.strftime("%Y-%m-%d %H:%M:%S")

        print(f"New message from {sender_id} at {timestamp}: {msg}")

        # Append message data to CSV
        with open(csv_file, 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow([timestamp, sender_id, msg])

    print("Bot is running and saving messages to", csv_file)
    await client.run_until_disconnected()

await main()


Bot is running and saving messages to flagged_messages.csv
New message from 8432945463 at 2025-07-28 16:55:39: hellooozzz
